In [8]:
import rasterio
import rasterio.warp
import matplotlib.pyplot as plt
import affine
import numpy
import pyproj
import pandas

import starepandas
import pystare

# Load Landsat Granule/Scene

In [9]:
fname = '../tests/data/landsat/LC08_L2SP_042036_20210729_20210804_02_T1_SR_B1.TIF'
with rasterio.open(fname) as dataset:
    data = dataset.read()
    mask = dataset.dataset_mask()
    crs = dataset.crs

    # Pixel center instead of pixel corner
    transform = dataset.transform * affine.Affine.translation(0.5, 0.5)

In [10]:
indices = numpy.indices(dataset.shape, dtype='int16')
xy = numpy.array(transform*indices, dtype='int32')

In [11]:
transformer = pyproj.Transformer.from_crs(crs, "epsg:4326")
lat, lon = transformer.transform(xy[0], xy[1])

In [ ]:
sids = pystare.from_latlon_2d(lat, lon, adapt_level=True)

In [ ]:
landsat = starepandas.STAREDataFrame({'lon': lon.flatten(),
                                      'lat': lat.flatten(),
                                      'band_1': data.flatten()})

In [ ]:
landsat = starepandas.STAREDataFrame(landsat, stare=sids.flatten())
landsat.to_pickle('landsat.pickle')

In [ ]:
landsat = pandas.read_pickle('landsat.pickle')

In [ ]:
del(xy)
del(lon)
del(lat)
del(sids)

In [ ]:
landsat#.head()

# Mod09

In [ ]:
fname = '../tests/data/mod05/MOD05_L2.A2021210.1935.061.2021213104049.hdf'
mod05 = starepandas.read_granule(fname, add_stare=True)
mod05 = mod05[['lat', 'lon', 'stare', 'Water_Vapor_Infrared']]
mod05.head()

In [ ]:
joined = starepandas.stare_join(landsat[0:100], mod05)
joined = starepandas.STAREDataFrame(joined)

In [ ]:
joined['hex_left'] = joined.set_stare('stare_left').hex()
joined['hex_right'] = joined.set_stare('stare_right').hex()

In [ ]:
joined.head()

In [ ]:
joined['trixels_right'] = joined.set_stare('stare_right').make_trixels()
joined['trixels_left'] = joined.set_stare('stare_left').make_trixels()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), dpi=300)
ax.grid(True)

joined.set_trixels('trixels_left').plot(ax=ax)
joined.set_trixels('trixels_right').plot(ax=ax, color='r')